# Analyzing Used Cars on eBay Kleinanzeigen

In this project, we'll work with a dataset of used cars from eBay Kleinanzeigen.

Over 370000 used cars were scraped with Scrapy from Ebay-Kleinanzeigen and uploded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data). Included fields are:

- *dateCrawled* : when this ad was first crawled, all field-values are taken from this date
- *name* : "name" of the car
- *seller* : private or dealer
- *offerType* : type of listing
- *price* : the price on the ad to sell the car
- *abtest* : whether the listing is included in an A/B test
- *vehicleType* : the vehicle type
- *yearOfRegistration* : at which year the car was first registered
- *gearbox* : the transmission type 
- *powerPS* : power of the car in PS
- *model* : the car model name
- *kilometer* : how many kilometers the car has driven
- *monthOfRegistration* : at which month the car was first registered
- *fuelType* : what type of fuel the car uses
- *brand* : the brand of the car
- *notRepairedDamage* : if the car has a damage which is not repaired yet
- *dateCreated* : the date for which the ad at ebay was created
- *nrOfPictures* : number of pictures in the ad 
- *postalCode* : the postal code for the location of the vehicle
- *lastSeenOnline* : when the crawler saw this ad last online

## Importing Libraries, Opening and Examination of the Data Set

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv", encoding = "Latin-1")

In [2]:
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType            333659 non-null object
yearOfRegistration     371528 non-null int64
gearbox                351319 non-null object
powerPS                371528 non-null int64
model                  351044 non-null object
kilometer              371528 non-null int64
monthOfRegistration    371528 non-null int64
fuelType               338142 non-null object
brand                  371528 non-null object
notRepairedDamage      299468 non-null object
dateCreated            371528 non-null object
nrOfPictures           371528 non-null int64
postalCode             371528 non-null int64
lastSeen              

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


We can make the following observations:

- The dataset contains 20 columns, strings and integers.
- Some columns have null values, but none have more than ~20% null values.
- The column names use camelcase instead of Python's preferred snakecase.

First, let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

## Cleaning Column Names

In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [4]:
# create new names for columns
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 
                 'abtest','vehicle_type', 'registration_year', 'gearbox', 
                 'power_ps', 'model', 'odometer', 'registration_month', 
                 'fuel_type', 'brand','unrepaired_damage', 'ad_created', 
                 'nr_of_pictures', 'postal_code','last_seen']

## Initial Data Exploration and Data Cleaning
We'll start by exploring the data to find obvious areas where we should clean the data.

In [5]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,371528,371528,371528,371528,3.715280e+05,371528,333659,371528.000000,351319,371528.000000,351044,371528.000000,371528.000000,338142,371528,299468,371528,371528.0,371528.00000,371528
unique,280500,233531,2,2,NaN,2,8,NaN,2,NaN,251,NaN,NaN,7,40,2,114,NaN,NaN,182806
top,2016-03-24 14:49:47,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-06 13:45:54
freq,7,657,371525,371516,NaN,192585,95894,NaN,274214,NaN,30070,NaN,NaN,223857,79640,263182,14450,NaN,NaN,17
mean,NaN,NaN,NaN,NaN,1.729514e+04,NaN,NaN,2004.577997,NaN,115.549477,NaN,125618.688228,5.734445,NaN,NaN,NaN,NaN,0.0,50820.66764,NaN
std,NaN,NaN,NaN,NaN,3.587954e+06,NaN,NaN,92.866598,NaN,192.139578,NaN,40112.337051,3.712412,NaN,NaN,NaN,NaN,0.0,25799.08247,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.00000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30459.00000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49610.00000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71546.00000,NaN


Our initial observations:
- There are some text columns where all (or nearly all) of the values are the same:
    - seller
    - offer_type
- The nr_of_pictures column looks odd, we'll need to investigate this further.

In [6]:
autos["seller"].value_counts()

privat        371525
gewerblich         3
Name: seller, dtype: int64

In [7]:
autos["nr_of_pictures"].value_counts()

0    371528
Name: nr_of_pictures, dtype: int64

These columns don't give us any useful information for analysis and we can drop them.

In [8]:
autos = autos.drop(["seller", "offer_type", "nr_of_pictures"], axis=1)

## Exploring Price and Odometer

Let's continue exploring the data, specifically looking for data that doesn't look right. We'll start by analyzing the `odometer` and `price` columns:

- We will analyze the columns using minimum and maximum values and look for any values that look unrealistically high or low (outliers) that we might want to remove.

In [9]:
print("Minimum price is:",autos["price"].min())
print("Maximum price is:", autos["price"].max())

print("\nUnique values:\n", autos["price"].unique())
print("\nDescription:\n", autos["price"].describe())

Minimum price is: 0
Maximum price is: 2147483647

Unique values:
 [  480 18300  9800 ... 18429 24895 10985]

Description:
 count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64


In [10]:
# the most frequent prices
autos["price"].value_counts().head(10)

0       10778
500      5670
1500     5394
1000     4649
1200     4594
2500     4438
600      3819
3500     3792
800      3784
2000     3432
Name: price, dtype: int64

In [11]:
# frequency of the highest prices
autos["price"].value_counts().sort_index (ascending = False).head(80)

2147483647     1
99999999      15
99000000       1
74185296       1
32545461       1
27322222       1
14000500       1
12345678       9
11111111      10
10010011       1
10000000       8
9999999        3
3895000        1
3890000        1
2995000        1
2795000        1
1600000        2
1300000        1
1250000        2
1234566        1
1111111        2
1010010        1
1000000        5
999999        13
999990         1
911911         1
849000         1
820000         1
780000         1
745000         2
              ..
370000         2
368000         1
350000         4
349000         1
345000         1
323223         1
300000         1
299000         3
295000         1
294900         1
285000         1
284000         1
279000         1
275000         1
270000         1
265000         1
260000         3
259000         1
258000         1
257500         1
254900         1
250000         4
249000         1
245000         1
239000         1
238000         1
237500         1
235000        

In [12]:
# frequency of the lowest prices
autos["price"].value_counts().sort_index (ascending = True).head(10)

0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
Name: price, dtype: int64

As we see, `price` column has outliers: 10778 rows with a price 0 and some unrealisticaly high prices: 2147483647, 99999999, 99000000 etc. We will remove all rows containing outliers, such as prices:
- lower than \€100 
- higher than \€300 000 (as prices jump up unrealistically too high after 300 thousands)

In [13]:
autos = autos[(autos["price"] > 100) & (autos["price"] < 300000)]  
autos["price"].describe()

count    357053.000000
mean       5949.746673
std        8749.477204
min         101.000000
25%        1299.000000
50%        3100.000000
75%        7500.000000
max      299000.000000
Name: price, dtype: float64

In [14]:
print("In the column `odometer`:\nMinimum value is:",autos["odometer"].min())
print("Maximum value:", autos["odometer"].max())

print("\nUnique values:\n", autos["odometer"].unique())
print("\nDescription:\n", autos["odometer"].describe())

print("\nValue counts:\n",autos["odometer"].value_counts().head(10))

In the column `odometer`:
Minimum value is: 5000
Maximum value: 150000

Unique values:
 [150000 125000  90000  30000  70000   5000 100000  60000  20000  80000
  50000  40000  10000]

Description:
 count    357053.000000
mean     125806.084811
std       39572.798900
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer, dtype: float64

Value counts:
 150000    230767
125000     37166
100000     15332
90000      12306
80000      10860
70000       9640
60000       8575
50000       7514
40000       6309
30000       5905
Name: odometer, dtype: int64


We can see that the values in this field are rounded, which might indicate that sellers had to choose from pre-set options for this field. Additionally, there are too many cars (more than 50%) with 150 000 km, which probably means that it is the maximum value that can be chosen on the website and these cars have odometer with MORE than or equal to 150 000 km.

## Explore Date Columns
Let's now move on to the date columns and understand the date range the data covers.

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself:
- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

Right now, the `date_crawled`, `last_seen`, and `ad_created` columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. 

Let's first understand how the values in the three string columns are formatted:

In [15]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-24 11:52:17,2016-03-24 00:00:00,2016-04-07 03:16:57
1,2016-03-24 10:58:45,2016-03-24 00:00:00,2016-04-07 01:46:50
2,2016-03-14 12:52:21,2016-03-14 00:00:00,2016-04-05 12:47:46
3,2016-03-17 16:54:04,2016-03-17 00:00:00,2016-03-17 17:40:17
4,2016-03-31 17:25:20,2016-03-31 00:00:00,2016-04-06 10:17:21


You could notice that the first 10 characters represent the day. To understand the date range, we can extract just the date values:

In [16]:
autos["day_crawled"] = autos["date_crawled"].str[:10]
autos["day_ad_created"] = autos["ad_created"].str[:10]
autos["day_last_seen"] = autos["last_seen"].str[:10]

autos["day_crawled"].value_counts(normalize = True, dropna = False).sort_index()

2016-03-05    0.025576
2016-03-06    0.014452
2016-03-07    0.035597
2016-03-08    0.033468
2016-03-09    0.034118
2016-03-10    0.032656
2016-03-11    0.032737
2016-03-12    0.036356
2016-03-13    0.015824
2016-03-14    0.036325
2016-03-15    0.033390
2016-03-16    0.030214
2016-03-17    0.031654
2016-03-18    0.013107
2016-03-19    0.035241
2016-03-20    0.036311
2016-03-21    0.035631
2016-03-22    0.032460
2016-03-23    0.031964
2016-03-24    0.029870
2016-03-25    0.032765
2016-03-26    0.032020
2016-03-27    0.030231
2016-03-28    0.035059
2016-03-29    0.034101
2016-03-30    0.033569
2016-03-31    0.031875
2016-04-01    0.034199
2016-04-02    0.035160
2016-04-03    0.038905
2016-04-04    0.037678
2016-04-05    0.012763
2016-04-06    0.003120
2016-04-07    0.001602
Name: day_crawled, dtype: float64

Looks like the site was crawled daily over roughly a one month period in March and April 2016.

In [17]:
autos["day_ad_created"].value_counts(normalize = True, dropna = False).sort_index(ascending = True)

2014-03-10    0.000003
2015-03-20    0.000003
2015-06-11    0.000003
2015-06-18    0.000003
2015-08-07    0.000003
2015-08-10    0.000003
2015-09-04    0.000006
2015-09-09    0.000003
2015-10-14    0.000003
2015-11-02    0.000003
2015-11-08    0.000003
2015-11-10    0.000003
2015-11-12    0.000003
2015-11-13    0.000003
2015-11-17    0.000003
2015-11-23    0.000006
2015-11-24    0.000006
2015-12-05    0.000008
2015-12-06    0.000003
2015-12-17    0.000003
2015-12-27    0.000003
2015-12-28    0.000003
2015-12-30    0.000008
2016-01-02    0.000011
2016-01-03    0.000003
2016-01-06    0.000003
2016-01-07    0.000008
2016-01-08    0.000003
2016-01-10    0.000011
2016-01-13    0.000008
                ...   
2016-03-09    0.034157
2016-03-10    0.032539
2016-03-11    0.032765
2016-03-12    0.036194
2016-03-13    0.017090
2016-03-14    0.035266
2016-03-15    0.033314
2016-03-16    0.030438
2016-03-17    0.031250
2016-03-18    0.014012
2016-03-19    0.034264
2016-03-20    0.036404
2016-03-21 

There is a larger variety of ad created dates. Most fall within 1-2 months of the listing date, but some of them are older.

In [18]:
autos["day_last_seen"].value_counts(normalize = True, dropna = False).sort_index(ascending = True)

2016-03-05    0.001266
2016-03-06    0.004083
2016-03-07    0.005181
2016-03-08    0.007898
2016-03-09    0.009786
2016-03-10    0.011458
2016-03-11    0.012959
2016-03-12    0.023249
2016-03-13    0.008425
2016-03-14    0.012177
2016-03-15    0.016258
2016-03-16    0.016440
2016-03-17    0.028752
2016-03-18    0.006893
2016-03-19    0.016337
2016-03-20    0.019860
2016-03-21    0.020014
2016-03-22    0.020442
2016-03-23    0.017981
2016-03-24    0.019157
2016-03-25    0.018961
2016-03-26    0.015944
2016-03-27    0.016642
2016-03-28    0.022176
2016-03-29    0.023257
2016-03-30    0.023672
2016-03-31    0.024249
2016-04-01    0.023949
2016-04-02    0.024949
2016-04-03    0.025330
2016-04-04    0.025489
2016-04-05    0.127048
2016-04-06    0.219169
2016-04-07    0.130552
Name: day_last_seen, dtype: float64

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [19]:
autos["registration_year"].describe()

count    357053.000000
mean       2004.328932
std          73.237695
min        1000.000000
25%        1999.000000
50%        2004.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64

Looking at this column, we note some odd values. The minimum value is 1000, long before cars were invented and the maximum is 9999, many years into the future.

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

In [20]:
autos = autos[(autos["registration_year"] > 1900) & (autos["registration_year"] < 2017)]

In [21]:
autos["registration_year"].describe()

count    343197.00000
mean       2002.94171
std           7.18039
min        1910.00000
25%        1999.00000
50%        2003.00000
75%        2008.00000
max        2016.00000
Name: registration_year, dtype: float64

In [22]:
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.065691
1999    0.063567
2005    0.062769
2006    0.058174
2001    0.057227
2003    0.056985
2004    0.056618
2002    0.054665
2007    0.050886
1998    0.049438
Name: registration_year, dtype: float64

We see that most of the vehicles were first registered between 1998 and 2007.

## Exploring Price by Brand

In order to explore `brand` column, we will use technique called `aggregation`:

- Identify the unique values we want to aggregate by
- Create an empty dictionary to store our aggregate data
- Loop over the unique values, and for each:
    - Subset the dataframe by the unique values
    - Calculate the mean of whichever column we're interested in
    - Assign the val/mean to the dict as k/v.

In [23]:
autos["brand"].value_counts()

volkswagen        72690
bmw               37823
opel              36358
mercedes_benz     33413
audi              30873
ford              23485
renault           16227
peugeot           10352
fiat               8781
seat               6402
skoda              5423
mazda              5287
smart              4946
citroen            4776
nissan             4668
toyota             4472
hyundai            3439
mini               3245
volvo              3148
sonstige_autos     3133
mitsubishi         2825
honda              2593
kia                2382
suzuki             2178
alfa_romeo         2158
porsche            2121
chevrolet          1734
chrysler           1321
dacia               863
jeep                755
land_rover          751
daihatsu            727
subaru              718
jaguar              594
saab                505
daewoo              493
trabant             461
lancia              440
rover               431
lada                206
Name: brand, dtype: int64

In [24]:
brand_counts = autos["brand"].value_counts()
top_brands = brand_counts[brand_counts > 4000].index
print(top_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'mazda', 'smart', 'citroen',
       'nissan', 'toyota'],
      dtype='object')


4 out of 5 top brands are German manufacturers. Volkswagen is a leader, with significant difference from the second brand BMW.

In [25]:
brand_mean_prices = {}
for brand in top_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_price = brand_only["price"].mean()
    brand_mean_prices[brand] = int(mean_price)
    
bmp_series = pd.Series(brand_mean_prices)
bmp_series

volkswagen       5451
bmw              8508
opel             3014
mercedes_benz    8581
audi             9125
ford             3759
renault          2473
peugeot          3298
fiat             2933
seat             4588
skoda            6548
mazda            4111
smart            3648
citroen          3780
nissan           4754
toyota           5352
dtype: int64

As we see, from our top 5 brands:

- `Audi`,`Mercedes-Benz` and `BMW` are much more expensive brands. 
- `Opel` is the cheapest one out of 5
- `Volkswagen` is in between, that might explain its popularity.

## Exploring Mileage

In [26]:
brand_mean_mileage = {}

for brand in top_brands:
    brand_only = autos[autos["brand"] == brand]
    mean_mileage = brand_only["odometer"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)
    
mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

brand_info = pd.DataFrame(mean_mileage,columns=['mean_mileage'])
brand_info["mean_price"] = mean_prices
brand_info

,mean_mileage,mean_price
bmw,133009,8508
mercedes_benz,130739,8581
audi,129563,9125
opel,128769,3014
volkswagen,128499,5451
renault,127891,2473
mazda,125747,4111
peugeot,124433,3298
ford,123715,3759
seat,120508,4588


There is not big difference in mean mileage of the top 5 brands, although  there is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.

## Conclusions

We cleaned and analysed a dataset of used cars from eBay Kleinanzeigen.
We found out that most of the cars (more than 75%):
- are high mileage cars (with odometer more than 125 thousand km)
- cost less than 7500 euros
- were first registered between 1998 and 2007

And our brand exploration showed that:
- German manufacturers represent four out of the top five brands, almost 50% of the overall listings.
- Volkswagen is a leader, with significant difference from the second brand BMW.
- Of the top 5 brands, there is a distinct price gap:
    - Audi, BMW and Mercedes Benz are much more expensive
    - Opel is the cheapest out of top 5
    - Volkswagen is in between - this may explain its popularity.
- The range of car mileages does not vary as much as the prices do by brand, although there is a slight trend to the more expensive vehicles having higher mileage, with the less expensive vehicles having lower mileage.